In [1]:
#Импортируем нужные библиотеки
import pandas as pd
import os

In [2]:
#Создаем функцию для загрузки файла, которая примет имя файла и промежуток столбцов, который нужен нам
def load_data(filename, range):
    df = pd.read_excel(f"{os.getcwd()}/{filename}", header=3, usecols=range)
    first_row_with_all_NaN = df[df.isnull().all(axis=1) == True].index.tolist()[0]
    df = df.loc[0:first_row_with_all_NaN-1]
    return df


In [3]:
#Здесь мы исправляем ошибки, как лишние пробелы, в т.ч. неразрывные пробелы
name = "mapping_тестовое задание.xlsx"
df1 = load_data(name, "A:E")
df1["name"] = df1['first_name'] + ' ' + df1['last_name'] + ' ' + df1['surname']
df1['name'] = df1['name'].str.replace(r'\s+', ' ', regex=True)
df2 = load_data(name, "G:I")
df2.rename(columns={"company.1":"company"}, inplace=True)
df2['name'] = df2['name'].str.replace('\xa0', ' ')
df2['name'] = df2['name'].str.replace(r'\s+', ' ', regex=True)

In [4]:
#Это функция для сопоставления id_2 с именами и компаниями, принимающая одну переменную
def get_id_2(row):
    name = row['name']
    company = row['company']
    
    matches = df2[(df2['company'] == company) & (df2['name'].str.lower() == name.lower())]['id_2'].unique()
    
    if len(matches) == 1:
        return matches[0]
    else:
        return ""
#Здесь мы применяем эту функцию
df1['id_2'] = df1.apply(get_id_2, axis=1)

In [5]:
#Это функция для генерации комментов, тоже принимающая одну переменную
def get_id_2_comments(row):
    name = row['name']
    company = row['company']
    
    matches = df2[(df2['company'] == company) & (df2['name'].str.lower() == name.lower())]['id_2'].unique()
    
    if len(matches) == 0:
        return "Подходящей записи в source_2 не найдено"
    elif len(matches) > 1:
        return f"Невозможно установить однозначное соответствие. В source_2 {len(matches)} подходящих записи с id_2: {', '.join(map(str, matches))}"
    else:
        return ""
#Здесь мы применяем эту функцию
df1['comments'] = df1.apply(get_id_2_comments, axis=1)

In [6]:
#Удаляем именную колонку, созданную ранее и сохраняем данные в эксель
df1.drop(columns=['name'], inplace=True)
df1.to_excel(f"{os.getcwd()}/output.xlsx")
